## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*2


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([8,13])
x = np.repeat(x,26)
y = twentysix*2

## Creating X and Y values for paths

In [3]:
# X values
xvals = [[8,13]]*26

In [4]:
# Y values
# rotor 2
yvals = [[1,1],[2,10],[3,4],[4,11],[5,19],[6,9],[7,18],[8,21],[9,24],
    [10,2],[11,12],[12,8],[13,23],[14,20],[15,13],[16,3],[17,17],
    [18,7],[19,26],[20,14],[21,16],[22,25],[23,6],[24,22],[25,15],
    [26,5]]

# Rotor Links - Rotor 2 

In [5]:
# establish links
links = {
    0:[26],     1:[35],      2:[29],     3:[36],     4:[44],     5:[34],      
    6:[43],     7:[46],      8:[49],     9:[27],    10:[37],    11:[33],   
    12:[48],    13:[45],    14:[38],    15:[28],    16:[42],    17:[32],    
    18:[51],    19:[39],    20:[41],    21:[50],    22:[31],    23:[47],    
    24:[40],    25:[30], 
        
    26:[0],     27:[9],     28:[15],    29:[2],     30:[25],    31:[22],
    32:[17],    33:[11],    34:[5],     35:[1],     36:[3],     37:[10],
    38:[14],    39:[19],    40:[24],    41:[20],    42:[16],    43:[6],
    44:[4],     45:[13],    46:[7],     47:[23],    48:[12],    49:[8],  
    50:[21],    51:[18]
}

## Create Graph and Add Features

In [6]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Hover over points')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [7]:
show(p)